In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
import nltk
import os
import pickle

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

SQL CODE:

SELECT *
FROM Posts
WHERE
    PostTypeId = 1 AND
    Score >=0 AND
    CreationDate >= '20180101' AND
    Tags <> 0
    

In [2]:
#dataloading
postsdf = pd.read_csv('posts with tags.csv',  index_col=0, parse_dates=['CreationDate', 'DeletionDate', 'LastEditDate', 'LastActivityDate', 'ClosedDate'])
postsdf.head()

,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,DeletionDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,...,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate
Id,,,,,,,,,,,,,,,,,,,,,
48046688,1,NaN,NaN,2018-01-01 01:43:05,NaT,0,32,<p>Hi I was trying to install a python package...,9159552.0,NaN,...,NaN,2018-01-01 02:50:41,2018-01-01 02:50:41,tried to install a pythonn package but encount...,<python><gcc>,0,0,NaN,NaT,NaN
48046690,1,NaN,NaN,2018-01-01 01:44:46,NaT,0,39,<p>I am trying to make a chrome extension that...,9159517.0,NaN,...,NaN,2018-01-01 16:59:50,2018-01-01 16:59:50,Update chrome extension popup text,<javascript><html><google-chrome-extension>,0,3,NaN,NaT,NaN
48046378,1,NaN,NaN,2018-01-01 00:00:29,NaT,0,39,<p>Calling a <em>dll</em> from Python using <e...,8276973.0,NaN,...,NaN,2018-01-01 03:16:54,2018-01-03 08:21:23,Python ctypes: how to pass ctypes array to DLL?,<python><ctypes>,2,0,NaN,NaT,NaN
48046379,1,NaN,NaN,2018-01-01 00:00:32,NaT,0,390,<p>I have a Django app with which users can cr...,1745384.0,NaN,...,NaN,2018-01-01 03:18:33,2018-01-11 23:14:31,nginx gunicorn 502 bad gateway: upstream prema...,<python><django><nginx>,1,0,1.0,NaT,NaN
48046382,1,NaN,NaN,2018-01-01 00:00:52,NaT,0,44,<p>I'm trying to implement hittest that is use...,8794406.0,NaN,...,NaN,NaT,2018-01-01 00:00:52,Implementing hit test,<c++><opencv><3d><computer-vision><arkit>,0,0,NaN,NaT,NaN


In [3]:
postsdf.columns.values

array(['PostTypeId', 'AcceptedAnswerId', 'ParentId', 'CreationDate',
       'DeletionDate', 'Score', 'ViewCount', 'Body', 'OwnerUserId',
       'OwnerDisplayName', 'LastEditorUserId', 'LastEditorDisplayName',
       'LastEditDate', 'LastActivityDate', 'Title', 'Tags', 'AnswerCount',
       'CommentCount', 'FavoriteCount', 'ClosedDate', 'CommunityOwnedDate'], dtype=object)

In [4]:
postsdf[postsdf['PostTypeId'] !=1]

,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,DeletionDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,...,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate
Id,,,,,,,,,,,,,,,,,,,,,


In [5]:
#What are the tags in the body ?
postsdf['TagsSet'] = postsdf['Body'].apply(lambda x: set([tag.name for tag in BeautifulSoup(x, 'lxml').find_all()]))
postsdf['TagsSetLen'] = postsdf['TagsSet'].apply(lambda x: len(x))
print(postsdf['TagsSetLen'].describe())
#print(postsdf['TagsSet'].str.join(sep='*').str.get_dummies(sep='*').describe())
postsdf[['TagsSet', 'Body', 'TagsSetLen']].head()

count    50000.000000
mean         5.413480
std          1.407265
min          3.000000
25%          5.000000
50%          5.000000
75%          6.000000
max         15.000000
Name: TagsSetLen, dtype: float64


,TagsSet,Body,TagsSetLen
Id,,,
48046688,"{body, html, p, code, pre}",<p>Hi I was trying to install a python package...,5
48046690,"{body, strong, html, p, code, pre}",<p>I am trying to make a chrome extension that...,6
48046378,"{body, blockquote, em, html, p, code, pre}",<p>Calling a <em>dll</em> from Python using <e...,7
48046379,"{body, html, p, code, pre}",<p>I have a Django app with which users can cr...,5
48046382,"{a, body, html, p, code, pre}",<p>I'm trying to implement hittest that is use...,6


In [6]:
postsdf['TagsSet'].str.join(sep='*').str.get_dummies(sep='*').columns.values

array(['a', 'b', 'blockquote', 'body', 'br', 'code', 'dd', 'div', 'dl',
       'dt', 'em', 'h1', 'h2', 'h3', 'hr', 'html', 'i', 'img', 'kbd', 'li',
       'ol', 'p', 'pre', 's', 'strike', 'strong', 'sub', 'sup', 'ul'], dtype=object)

In [7]:
#General parsing test

sampleline = 3

textsample = postsdf['Body'].iloc[sampleline]
titlesample = postsdf['Title'].iloc[sampleline]
tagssample = postsdf['Tags'].iloc[sampleline]
soupBody = BeautifulSoup(textsample, 'lxml')
soupTitle = BeautifulSoup(titlesample, 'lxml')
soupTags = BeautifulSoup(tagssample, 'html.parser')

In [8]:
#Tags parsing
'|'.join(set([tag.name for tag in soupTags.find_all()]))

'nginx|python|django'

In [9]:
#Code parsing
for text in soupBody.find_all('code'):
    print(text.text)

client_max_body_size 100M

fastcgi_buffers 8 16k;
fastcgi_buffer_size 32k;
fastcgi_connect_timeout 3000;
fastcgi_send_timeout 3000;
fastcgi_read_timeout 3000;

2017/12/31 23:50:51 [error] 1279#1279: *1 upstream prematurely closed connection while reading response header from upstream, 
client: 107.205.110.154, 
server: movingcollage.com, 
request: "POST /create-collage/ HTTP/1.1", 
upstream: "http://unix:/home/mike/movingcollage/movingcollage.sock:/create-collage/", 
host: "movingcollage.com", referrer: "http://movingcollage.com/create-collage/"



In [10]:
#Text parsing
for text in soupBody.find_all('p'):
    print(text.text)

I have a Django app with which users can create video collages using multiple videos. Problem is, on production, when uploading videos to amazon s3, I get a 502 bad gateway (works fine locally). Does anyone know what could be wrong? I already set 
and 
Does anyone know what could be wrong? Thanks in advance
Full error: 


In [11]:
#titleparsing
soupTitle.get_text()

'nginx gunicorn 502 bad gateway: upstream prematurely closed connection while reading response header from upstream'

In [12]:
#Tags parsing on dataset
postsdf['PTags'] = postsdf['Tags'].apply(lambda x: [tag.name for tag in BeautifulSoup(x, 'html.parser').find_all()])
postsdf[['PTags', 'Tags']].head()

,PTags,Tags
Id,,
48046688,"[python, gcc]",<python><gcc>
48046690,"[javascript, html, google-chrome-extension]",<javascript><html><google-chrome-extension>
48046378,"[python, ctypes]",<python><ctypes>
48046379,"[python, django, nginx]",<python><django><nginx>
48046382,"[c++, opencv, computer-vision, arkit]",<c++><opencv><3d><computer-vision><arkit>


In [13]:
#text parsing
postsdf['PTtext'] = postsdf['Body'].apply(lambda x: ' '.join([text.text for text in BeautifulSoup(x, 'lxml').find_all('p')]))
postsdf['PCode'] = postsdf['Body'].apply(lambda x: ' '.join([text.text for text in BeautifulSoup(x, 'lxml').find_all('code')]))
postsdf[['PTtext', 'PCode', 'Body']].head()

,PTtext,PCode,Body
Id,,,
48046688,Hi I was trying to install a python package ca...,pip install pysptk\n /usr/bin/ld: cannot find ...,<p>Hi I was trying to install a python package...
48046690,I am trying to make a chrome extension that di...,"<script src=""popup.js""></script>\n<html>\n<hea...",<p>I am trying to make a chrome extension that...
48046378,"Calling a dll from Python using ctypes, I want...",ReturnVec = ctypes.c_float * len(arrA)\nt = ty...,<p>Calling a <em>dll</em> from Python using <e...
48046379,I have a Django app with which users can creat...,client_max_body_size 100M\n fastcgi_buffers 8 ...,<p>I have a Django app with which users can cr...
48046382,I'm trying to implement hittest that is used b...,"std::pair<bool, double> linePlaneIntersection...",<p>I'm trying to implement hittest that is use...


In [14]:
#Review all the results
postsdf[['PTtext', 'PCode', 'Body', 'PTags', 'Tags']].head()

,PTtext,PCode,Body,PTags,Tags
Id,,,,,
48046688,Hi I was trying to install a python package ca...,pip install pysptk\n /usr/bin/ld: cannot find ...,<p>Hi I was trying to install a python package...,"[python, gcc]",<python><gcc>
48046690,I am trying to make a chrome extension that di...,"<script src=""popup.js""></script>\n<html>\n<hea...",<p>I am trying to make a chrome extension that...,"[javascript, html, google-chrome-extension]",<javascript><html><google-chrome-extension>
48046378,"Calling a dll from Python using ctypes, I want...",ReturnVec = ctypes.c_float * len(arrA)\nt = ty...,<p>Calling a <em>dll</em> from Python using <e...,"[python, ctypes]",<python><ctypes>
48046379,I have a Django app with which users can creat...,client_max_body_size 100M\n fastcgi_buffers 8 ...,<p>I have a Django app with which users can cr...,"[python, django, nginx]",<python><django><nginx>
48046382,I'm trying to implement hittest that is used b...,"std::pair<bool, double> linePlaneIntersection...",<p>I'm trying to implement hittest that is use...,"[c++, opencv, computer-vision, arkit]",<c++><opencv><3d><computer-vision><arkit>


In [15]:
#Number  cleaning,
def number_cleaning(tokenized):
    toreturn = []
    for word in tokenized:
        temp = word.translate(str.maketrans(dict([(str(x), None) for x in range(10)])))
        if len(temp) != 0:
            toreturn.append(temp)
    return toreturn
number_cleaning(['ab1 2cd34e5', '123', 'qwerty'])

['ab cde', 'qwerty']

In [16]:
#word stemming
stemmer = SnowballStemmer("english")
def wordstemmer(word):
    return stemmer.stem(word)

wordstemmer('bones')

'bone'

In [17]:
def customtokenizer(entry):
    tokenizer = RegexpTokenizer(r'\w+')
    toreturn = tokenizer.tokenize(entry)
    toreturn = number_cleaning(toreturn)
    toreturn = [wordstemmer(w) for w in toreturn]
    return toreturn
customtokenizer('Ceci est un test !! 1, 2ew 47 et 3 :-)')
customtokenizer(''' It's official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria.''')

['it',
 's',
 'offici',
 'u',
 's',
 'presid',
 'barack',
 'obama',
 'want',
 'lawmak',
 'to',
 'weigh',
 'in',
 'on',
 'whether',
 'to',
 'use',
 'militari',
 'forc',
 'in',
 'syria']

In [23]:
def cvect(corpus, stop_words, max_df):
    cvec = CountVectorizer(lowercase=True, tokenizer=customtokenizer, token_pattern=r'\w+', stop_words=stop_words, max_df=max_df, max_features=500)
    term_freq = cvec.fit_transform(corpus)
    zipped = zip([k for k in sorted(cvec.vocabulary_, key=cvec.vocabulary_.get, reverse=False)], np.asarray(term_freq.sum(axis=1)).reshape(1,-1)[0])
    vocabulary = [x for x in zipped]
    return term_freq, vocabulary

In [24]:
sw_articles = set(nltk.corpus.stopwords.words('english'))
max_df_articles = 0.8
Posts_freq, Post_vocabulary = cvect(postsdf['PTtext'], sw_articles, max_df_articles)

In [25]:
articles_tfidf = TfidfTransformer().fit_transform(Posts_freq)
articles_tfidf.toarray()

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [26]:
Post_vocabulary

[('abl', 40),
 ('abov', 151),
 ('access', 31),
 ('account', 31),
 ('achiev', 16),
 ('action', 36),
 ('activ', 21),
 ('actual', 57),
 ('ad', 29),
 ('add', 46),
 ('address', 44),
 ('advanc', 17),
 ('allow', 39),
 ('alreadi', 25),
 ('also', 23),
 ('alway', 8),
 ('android', 46),
 ('angular', 28),
 ('ani', 9),
 ('anoth', 16),
 ('answer', 32),
 ('anyon', 25),
 ('anyth', 27),
 ('apach', 52),
 ('api', 26),
 ('app', 24),
 ('appear', 33),
 ('appli', 40),
 ('applic', 78),
 ('appreci', 35),
 ('approach', 27),
 ('argument', 53),
 ('around', 31),
 ('array', 30),
 ('ask', 71),
 ('assign', 35),
 ('attempt', 41),
 ('attribut', 3),
 ('authent', 25),
 ('automat', 38),
 ('avail', 23),
 ('b', 19),
 ('back', 24),
 ('background', 35),
 ('bar', 17),
 ('base', 85),
 ('basic', 65),
 ('becaus', 62),
 ('befor', 75),
 ('best', 39),
 ('better', 93),
 ('bit', 48),
 ('block', 13),
 ('box', 9),
 ('browser', 16),
 ('build', 61),
 ('button', 33),
 ('c', 8),
 ('calcul', 22),
 ('call', 78),
 ('cannot', 44),
 ('case', 33),

In [22]:
tokenizer = RegexpTokenizer(r'\w+')
tokenizer.tokenize('Ceci est un test !!')

['Ceci', 'est', 'un', 'test']